In [4]:
import sys
import os

# Ensure project root is in sys.path for imports
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)


In [5]:
from crypto.shamir import generate_shares, reconstruct_secret
from crypto.schnorr import generate_keypair, sign_message, verify_signature


In [6]:
# Generate a master Schnorr private key and corresponding public key
priv, pub = generate_keypair()
print("Master Private Key:", priv)
print("Public Key:", pub)

# Share the private key among n signers with threshold t
n = 5
t = 3
shares = generate_shares(priv, n, t)
print("\nGenerated Shares:")
for idx, (x, y) in enumerate(shares, 1):
    print(f"  Signer {idx}: (x={x}, share={y})")


Master Private Key: 94918042472323276555976521889489848202446909001773631448924720679377578922040
Public Key: 106731055654740968458556655719401105348452329970044290626325787848938908472102

Generated Shares:
  Signer 1: (x=1, share=121940817164784644403366454393422157003464301490560366259661494458590558810910)
  Signer 2: (x=2, share=29050266259910320852677750193753744484126819484362541542913464659847087862968)
  Signer 3: (x=3, share=24598007073791547138237155602609058895670025209650648812866962500197436538695)
  Signer 4: (x=4, share=108584039606428323260044670619988100238093918666424688069521987979641604838091)
  Signer 5: (x=5, share=72656746541729407983773548933766420260162937628214167798692209881129322300675)


In [7]:
# Choose t signers (simulate: first t signers participate)
participating_shares = shares[:t]
print("\nParticipating Shares (for threshold):")
for x, y in participating_shares:
    print(f"  x={x}, share={y}")

# Reconstruct the private key from t shares (as done in threshold Schnorr)
recovered_priv = reconstruct_secret(participating_shares)
print("\nRecovered Private Key from shares:", recovered_priv)
assert recovered_priv == priv



Participating Shares (for threshold):
  x=1, share=121940817164784644403366454393422157003464301490560366259661494458590558810910
  x=2, share=29050266259910320852677750193753744484126819484362541542913464659847087862968
  x=3, share=24598007073791547138237155602609058895670025209650648812866962500197436538695

Recovered Private Key from shares: 94918042472323276555976521889489848202446909001773631448924720679377578922040


In [8]:
message = "Approve transaction X for £100,000"

# Use the reconstructed private key to sign
r, s = sign_message(recovered_priv, message)
print("\nSignature: (r, s) =", (r, s))

# Verify the signature with the public key
is_valid = verify_signature(pub, message, r, s)
print("Signature valid?", is_valid)



Signature: (r, s) = (35531033325588371674880218436380623569834557439451068871988148721699158019574, 37656692145183035872050977384608036835526693586964726543472958741185550482654)
Signature valid? True


In [9]:
# Try reconstructing with a different combination of shares
import itertools

for share_combo in itertools.combinations(shares, t):
    reconstructed = reconstruct_secret(list(share_combo))
    assert reconstructed == priv, "Failed reconstruction with valid threshold shares!"

print("\nAll combinations of t shares successfully reconstruct the secret!")



All combinations of t shares successfully reconstruct the secret!
